In [1]:
%pylab inline

Populating the interactive namespace from numpy and matplotlib


In [2]:
from importlib import reload
import os, sys, re, glob, time, pickle, IPython, logging
import scipy.ndimage as ndi
from itertools import islice
import torch
from torch import nn, optim
from torch.nn import functional as F
from torchmore import layers, flex
import torchtrainers as tt
from torch.utils.data import DataLoader
from webdataset import WebDataset
from ocrlib import ocrhelpers as helpers
from ocrlib.ocrhelpers import *
from ocrlib import ocrmodels as models
RUN("date"); RUN("hostname"); RUN("whoami"); RUN("nvidia-smi -L")

charset = "ABCDEFGHIJKLMNOPQRSTUVWXYZabcdefghijklmnopqrstuvwxyz"

def encode_str(s): 
    return [charset.find(c)+1 for c in s]

def decode_str(l): 
    return "".join([charset[k-1] for k in l])

transforms = [
    lambda x: (torch.tensor(x).float()/255.0).unsqueeze(0),
    lambda s: torch.tensor(encode_str(s)).long()
]
training = WebDataset("data/words-simple-training.tar", decoder="l8", 
                      extensions="jpg;jpeg;ppm;png txt", transforms=transforms)
testing = WebDataset("data/words-simple-test.tar", decoder="l8", 
                     extensions="jpg;jpeg;ppm;png txt", transforms=transforms)
training_dl = DataLoader(training, batch_size=5, collate_fn=helpers.collate4ocr)
testing_dl = DataLoader(testing, batch_size=20, collate_fn=helpers.collate4ocr)
next(iter(training_dl))[0].size()

IndentationError: unindent does not match any outer indentation level (ocrhelpers.py, line 262)

In [ ]:
def compute_moments(image):
    assert amin(image) >= 0
    d0, d1 = image.shape
    c0, c1 = mgrid[0:d0, 0:d1]
    m0 = sum(c0*image) / sum(image)
    m1 = sum(c1*image) / sum(image)
    m00 = sum((c0-m0)*(c0-m0)*image) / sum(image)
    m01 = sum((c0-m0)*(c1-m1)*image) / sum(image)
    m11 = sum((c1-m1)*(c1-m1)*image) / sum(image)
    return array([m0, m1]), array([[m00, m01], [m01, m11]])

def normalize(image, height=80, r=5):
    assert amin(image) >= 0 and amax(image) <= 1
    h, w = image.shape
    (y, x), ((m00, m01), (m10, m11)) = compute_moments(image > amax(image)*0.5)
    scale = 2.0*r*(m00**.5) / height
    delta = y - scale*(height/2.0)
    result = ndi.affine_transform(image, scale*eye(2), 
                                  order=1,
                                  offset=(delta, 0), 
                                  output_shape=(height, int(w/scale)))
    assert result.shape[0] == height, result.shape
    return result

image = zeros((100, 400))
image[10:20,10:300] = 1.0
normalized = normalize(image)
subplot(211); imshow(image)
subplot(212); imshow(normalized)
#normalize(normalized);

In [ ]:
transforms = [
    lambda x: torch.tensor(normalize(x/255.0)).unsqueeze(0),
    lambda s: torch.tensor(encode_str(s)).long()
]

training = WebDataset("data/words-simple-training.tar", decoder="l8", 
                      extensions="jpg;jpeg;ppm;png txt", transforms=transforms)
testing = WebDataset("data/words-simple-test.tar", decoder="l8", 
                     extensions="jpg;jpeg;ppm;png txt", transforms=transforms)
training_dl = DataLoader(training, batch_size=5,
                         collate_fn=helpers.collate4ocr)
testing_dl = DataLoader(testing, batch_size=20,
                        collate_fn=helpers.collate4ocr)

In [ ]:
figsize(12, 8)
for i, sample in enumerate(islice(training_dl, 0, 60, 10)):
    subplot(6, 2, i+1)
    imshow(sample[0][0,0].detach().numpy())
    title(str(sample[0].shape))

In [ ]:
!awk '/lstm_normalized/' RS="\n\n" ocrlib/ocrmodels.py

In [ ]:
model = models.make("lstm_normalized")
model

In [ ]:
trainer = helpers.LineTrainer(model)

In [ ]:
trainer.train(training_dl, 10, every=10)

In [ ]:
figsize(10, 10)
for i, batch in enumerate(islice(training_dl, 0, 10)):
    subplot(5, 2, i+1)
    result = trainer.predict_batch(*batch[:2])
    imshow(batch[0][0,0].detach().numpy())
    title(decode_str(result[0]))

In [ ]:
trainer.errors(testing_dl)